### Getting ready to train on 8xH100

I'm requesting the "right" to use 8xH100 on paperspace on a Sat afternoon. Not sure how long it will take.

No response as of Sunday morning. I see Lambda is cheaper. See what it looks like. Play with that in `trying-labmda-cloud.ipynb`.

#### add special tokens to tokenizer

Let me adjust the tokenizer to include the special tokens. Even though I won't need them for pretrain, I hope not to need to repeat this (expensive and time consuming) pretrain with a different vocab.

Try it:

In [3]:
import os
os.environ["PYTHONPATH"] = "../my_nanochat"

In [4]:
!python -m scripts.my_tok_train

max_chars: 10,000,000,000
doc_cap: 10,000
vocab_size: 65,536
starting to train tokenizer
buffers filled: 1
buffers filled: 2
buffers filled: 3
buffers filled: 4
buffers filled: 5
buffers filled: 6
buffers filled: 7
buffers filled: 8
buffers filled: 9
buffers filled: 10
buffers filled: 11
buffers filled: 12
buffers filled: 13
buffers filled: 14
buffers filled: 15
buffers filled: 16
buffers filled: 17
buffers filled: 18
buffers filled: 19
buffers filled: 20
buffers filled: 21
buffers filled: 22
buffers filled: 23
buffers filled: 24
buffers filled: 25
buffers filled: 26
buffers filled: 27
buffers filled: 28
buffers filled: 29
buffers filled: 30
buffers filled: 31
buffers filled: 32
buffers filled: 33
buffers filled: 34
buffers filled: 35
buffers filled: 36
buffers filled: 37
buffers filled: 38
buffers filled: 39
buffers filled: 40
buffers filled: 41
buffers filled: 42
buffers filled: 43
buffers filled: 44
buffers filled: 45
buffers filled: 46
buffers filled: 47
buffers filled: 48
buffers 

In [5]:
import sys
sys.path.append('../my_nanochat')
from my_nanochat.my_tokenizer import get_tokenizer

In [6]:
tokenizer = get_tokenizer()

In [7]:
tokenizer.get_vocab_size()

65536

In [8]:
tokenizer.get_special_tokens()

{'<|assistant_end|>',
 '<|assistant_start|>',
 '<|bos|>',
 '<|output_end|>',
 '<|output_start|>',
 '<|python_end|>',
 '<|python_start|>',
 '<|user_end|>',
 '<|user_start|>'}

In [9]:
tokenizer.get_bos_token_id()

65527

In [11]:
tokenizer.encode_special('<|assistant_end|>')

65531

In [13]:
tokenizer.encode_special('<|foo|>') #expect error

KeyError: b'<|foo|>'

In [15]:
tokenizer.decode([65527, 65, 65531])

'<|bos|>A<|assistant_end|>'

### Instructions

Consolidating instructions here so I don't have to look back at challenge 14.

```
ssh paperspace@[public_ip]

# ssh key for git
ssh-keygen -t ed25519 -C "paperspace-vm"
cat ~/.ssh/id_ed25519.pub
copy into github UI (https://github.com/settings/keys)

git config --global user.email "ericsilberstein@gmail.com"
git config --global user.name "Eric Silberstein"

# clone this repo
git clone git@github.com:ericsilberstein1/nanogpt-learning.git

# Basic packages
sudo apt-get update && sudo apt-get install -y build-essential git curl wget ca-certificates

# confirm / install smi
nvidia-detector
nvidia-smi
sudo apt-get install nvidia-driver-580 nvidia-utils-580
sudo modprobe nvidia # (before did sudo shutdown -r now)
nvidia-smi

# later, after working on challenge 15, looked into why torch was failing to compile the model
# and realized needed to do this so python headers get installed
# the specific error was: /tmp/tmp0normshd/cuda_utils.c:6:10: fatal error: Python.h: No such file 
# or directory 6 | #include <Python.h>
sudo apt-get install python3.10-dev

# UV
curl -LsSf https://astral.sh/uv/install.sh | sh
source .zshrc

# rust
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh
echo '. "$HOME/.cargo/env"' >> .zshrc
source .zshrc

cd nanogpt-learning

uv sync
source .venv/bin/activate

# download data files
PYTHONPATH=my_nanochat python -m my_nanochat.my_dataset --num-files 8

# download another 240 in the background, check this completes before starting base_train
PYTHONPATH=my_nanochat python -m my_nanochat.my_dataset --num-files 240 &

# for now until organize this better
uv tool install maturin
cd challenge-07-rust-and-python-simplified-tokenizer/rust_tokenizer
maturin develop
cd -

# in challenge 23 I created my_tok_train.py so now can train the tokenizer here rather
# than from a notebook
PYTHONPATH=my_nanochat python -m scripts.my_tok_train

# while that's running

# in .zshrc add
# export WANDB_API_KEY="XXX"

uv run jupyter lab

# ON MY LAPTOP make port 8889 a tunnel to jupyter
ssh -N -L 8889:localhost:8888 paperspace@[public_ip]
```

In tmux shell:

```
source .venv/bin/activate

cd challenge-25-pretrain-d20

torchrun --standalone --nproc_per_node=8 -m scripts.my_base_train -- --depth=20 --run=challenge-25-1 > train_output_001.txt 2>&1
```